In [1]:
import datetime, time, math, os, json, shutil, asyncio, networkx as nx
from utils.gitutils import create_pull_request, clone_repo, create_branch, wipe_repo, prepare_repo
from utils.agent import Agent, GenerationConfig, Interaction, Team
from utils.stringutils import arr_from_sep_string, extract_markdown_blocks, markdown_to_dict
from utils.filetreeutils import FileTree, write_file_tree
from utils.listutils import flatten
from utils.frameworkutils import DartAnalyzer
from dotenv import load_dotenv
from utils.graphutils import loose_level_order, collapsed_level_order
from utils.frameworkutils import Framework

In [2]:
load_dotenv()  # Load environment variables from .env

GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
API_KEY = os.getenv('API_KEY')

In [3]:
repo_url = 'https://github.com/6165-MSET-CuttleFish/TeamTrack'
source = Framework.FLUTTER
feature = f"""# Name: Organizations
# Description: Organizations are a way to group users together. They can be used to automatically share events with the members of the organization whenever they are uploaded to the cloud. \
Organizations consist of a name, team number, and a backend-generated unique id. They can also have a profile picture, description, and location. \
Members can also opt to be publicly listed as a member of the organization or not. \
"""

In [4]:
# Clone the repo and make a new branch
repo = clone_repo(repo_url)
base_branch = repo.active_branch.name
created_branch = f"modification-{base_branch}"
local_repo_path = str(repo.working_dir)
working_dir_path = f'{local_repo_path}\\{source.get_working_dir()}'

In [5]:
source_swe_summarizer = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_analyzer",
    generation_config=GenerationConfig(temperature=0.7),
    system_prompt=f'''You are a {source} software engineer. Your job is to summarize the functionality of provided code files. Structure your response as follows:

# filename.ext:

$7 sentence summary of the functionality/contents of the file$
'''
)

pm = Agent(
    model_name="gemini-1.5-pro-001",
    api_key=API_KEY,
    name="pm",
    generation_config=GenerationConfig(temperature=0.3),
    system_prompt=f"""You are a high-level technical project manager tasked with the project of adding functionality to a {source} repository."""
)

source_swe_coder = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_coder",
    generation_config=GenerationConfig(temperature=0.7),
    system_prompt=f"You are a {source} software engineer. Your job is to write code to implement a specific feature. Respond with code and nothing else. No explanations needed."
)

team = Team(source_swe_summarizer, pm, source_swe_coder)

In [6]:
analyzer = source.get_analyzer(working_dir_path)

source_dependency_graph = analyzer.buildDependencyGraph()

In [7]:
# loose level order
eval_order = loose_level_order(source_dependency_graph)[::-1]
for level in eval_order:
    print(level)
print(f"Number of levels: {len(eval_order)}")

['models\\AppModel.dart', 'models\\StatConfig.dart']
['models\\GameModel.dart']
['models\\ScoreModel.dart']
['functions\\Extensions.dart', 'components\\misc\\PlatformGraphics.dart']
['components\\statistics\\BarGraph.dart', 'components\\users\\PFP.dart']
['functions\\Statistics.dart', 'providers\\Auth.dart', 'components\\scores\\ScoreSummary.dart', 'components\\users\\UsersRow.dart', 'components\\scores\\Incrementor.dart']
['views\\home\\match\\MatchView.dart']
['functions\\Functions.dart', 'components\\statistics\\PercentChange.dart']
['components\\scores\\ScoreRangeSummary.dart', 'models\\Change.dart', 'components\\scores\\ScoreTimeline.dart']
['views\\home\\match\\MatchRow.dart', 'components\\misc\\CardView.dart', 'components\\statistics\\CheckList.dart', 'views\\home\\match\\MatchConfig.dart', 'views\\home\\match\\ExampleMatchRow.dart', 'views\\home\\change\\ChangeConfig.dart', 'components\\scores\\ScoringElementStats.dart', 'views\\home\\change\\ChangeRow.dart', 'components\\misc\

In [8]:
(source_file_tree := FileTree.from_dir(working_dir_path))

├── api\
│   ├── README.MD
├── components\
│   ├── misc\
│   │   ├── CardView.dart
│   │   ├── Collapsible.dart
│   │   ├── EmptyList.dart
│   │   ├── InfoPills.dart
│   │   ├── PlatformGraphics.dart
│   ├── scores\
│   │   ├── Incrementor.dart
│   │   ├── ScoreCard.dart
│   │   ├── ScoreRangeSummary.dart
│   │   ├── ScoreSummary.dart
│   │   ├── ScoreTimeline.dart
│   │   ├── ScoringElementStats.dart
│   ├── statistics\
│   │   ├── BarGraph.dart
│   │   ├── CheckList.dart
│   │   ├── PercentChange.dart
│   ├── users\
│   │   ├── PFP.dart
│   │   ├── UsersRow.dart
│   ├── AutonomousDrawingTool.dart
├── functions\
│   ├── APIMethods.dart
│   ├── Extensions.dart
│   ├── Functions.dart
│   ├── GPTModel.dart
│   ├── ResponseModel.dart
│   ├── Statistics.dart
├── models\
│   ├── AppModel.dart
│   ├── Change.dart
│   ├── GameModel.dart
│   ├── GPTModel.dart
│   ├── ScoreModel.dart
│   ├── StatConfig.dart
├── providers\
│   ├── Auth.dart
│   ├── PushNotifications.dart
│   ├── Theme.dart
│   ├

In [9]:
# Summarize the original repo
from tqdm import tqdm

async def summarize_group(group):
    async def summarize(node):
        name = source_file_tree.nodes[node]['name']
        content = source_file_tree.nodes[node]['content']
        message = f"{name}\n```\n{content}\n```"
        return await team.async_chat_with_agent(
            agent_name='source_swe_analyzer',
            message=message,
            context_keys=[f"summary_{neighbor}" for neighbor in source_dependency_graph[node]],
            save_keys=[f"summary_{node}", "all"],
            prompt_title=f"Summary of {node}"
            )
    tasks = [summarize(node) for node in group]
    responses = await asyncio.gather(*tasks)
    for i, response in enumerate(responses):
        source_file_tree.nodes[group[i]]["summary"] = response

for level in tqdm(eval_order):
    await summarize_group(level)
    time.sleep(0.5)

  0%|          | 0/19 [00:00<?, ?it/s]

100%|██████████| 19/19 [01:20<00:00,  4.24s/it]


In [10]:
prompt = f"""This is the file tree for the original {source} repo:
```
{source.get_working_dir()}\\
{source_file_tree}
```
Given the prior context, summarize the functionality of the entire repository succinctly.
"""
print(team.chat_with_agent('source_swe_analyzer', prompt, context_keys=["all"], save_keys=["all"], prompt_title=f"""Repository Summary.
File Tree provided:
```
{source.get_working_dir()}\\
{source_file_tree}
```
"""))

This repository houses the source code for the TeamTrack Flutter application, a tool designed for managing and analyzing data from First Tech Challenge (FTC) robotics competitions. 

The app provides a comprehensive set of features, including:

* **Event Management:** Users can create, manage, and share events with other users. They can add teams, matches, and robot iterations to events, and view detailed performance statistics.
* **Team Management:** Users can view and analyze team performance data, including scores, rankings, and alliance recommendations. They can also draw autonomous paths and manage team changes.
* **Match Management:** Users can view match details, edit scores, control the match timer, and manage active users.
* **Alliance Management:** Users can simulate alliance formations, explore team compatibility, and receive AI-driven recommendations for alliance partners.
* **Inbox and Blocking:** Users can manage incoming event requests, block other users, and view their 

In [11]:
curr_string = '\n'.join([message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict()])
(curr_tokens := pm.model.count_tokens(curr_string).total_tokens)

20296

In [12]:
would_be_string = '\n'.join([source_file_tree.nodes[node]['content'] for node in source_dependency_graph.nodes])
(would_be_tokens := pm.model.count_tokens(would_be_string).total_tokens)

122951

In [13]:
(percent_tokens_reduction := 1 - (curr_tokens / would_be_tokens))

0.8349261087750405

In [14]:
response_format = f"""
You may choose ACTIONS from the following list:
- WRITE | $FILE_PATH$ | $PROMPT$ (PROMPT should inform how the new FILE_PATH should be written)
- EDIT | $FILE_PATH$ | $PROMPT$ (Edit the provided file to fulfill the prompt)
- DELETE | $FILE_PATH$ (Delete the provided file)
----------
Here is a sample response:
```
1. WRITE | {source.get_working_dir()}\\path\\to\\newfile.ext | Implement ... with the following properties:
    - property1
    - property2
2. EDIT | {source.get_working_dir()}\\path\\to\\file.ext | Fix the bug ... by doing:
    - step1
    - step2
3. DELETE | {source.get_working_dir()}\\path\\to\\some\\file.ext | Reasons for deletion:
    - reason1
    - reason2
```
"""

# Create new file tree
prompt = f'''
You are to build the following feature:
{feature}
------------------
{response_format}
------------------
You need to follow the response template structure exactly or the response will not be accepted.
'''
print(response := team.chat_with_agent("pm", prompt, context_keys=["all"], save_keys=["all"], prompt_title="Feature Creation Steps"))

```
1. WRITE | lib/models/Organization.dart | Implement the `Organization` class with the following properties:
    - `id`: String, backend-generated unique ID
    - `name`: String, name of the organization
    - `teamNumber`: int, team number associated with the organization
    - `profilePicture`: String, URL of the organization's profile picture (optional)
    - `description`: String, description of the organization (optional)
    - `location`: String, location of the organization (optional)

2. WRITE | lib/providers/OrganizationProvider.dart | Implement the `OrganizationProvider` class using ChangeNotifier to manage organization data and interactions. Include functionalities for:
    - Creating a new organization.
    - Fetching organization details.
    - Updating organization information.
    - Listing organizations a user is a member of.
    - Managing organization members (adding, removing, updating member visibility).

3. WRITE | lib/views/organization/OrganizationSettings.dar

In [15]:
import re

pattern  = re.compile(r'^\d+\.\s*((?:.*\n?)*?(?=\n\d+\.|\Z))', re.MULTILINE)
md = extract_markdown_blocks(response)[0]
lines = pattern.findall(md)
actions = [(action[0].strip(), action[1].strip()[len(source.get_working_dir()) + 1:], action[2].strip()) for line in lines if (action := line.split('|'))]
for action in actions:
    print(action)

('WRITE', 'models/Organization.dart', "Implement the `Organization` class with the following properties:\n    - `id`: String, backend-generated unique ID\n    - `name`: String, name of the organization\n    - `teamNumber`: int, team number associated with the organization\n    - `profilePicture`: String, URL of the organization's profile picture (optional)\n    - `description`: String, description of the organization (optional)\n    - `location`: String, location of the organization (optional)")
('WRITE', 'providers/OrganizationProvider.dart', 'Implement the `OrganizationProvider` class using ChangeNotifier to manage organization data and interactions. Include functionalities for:\n    - Creating a new organization.\n    - Fetching organization details.\n    - Updating organization information.\n    - Listing organizations a user is a member of.\n    - Managing organization members (adding, removing, updating member visibility).')
('WRITE', 'views/organization/OrganizationSettings.dart

## Context Caching

In [16]:
source_swe_coder.model.count_tokens('\n'.join(message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict())).total_tokens

20885

In [17]:
# Cache the context threads
factor = 32768 / source_swe_coder.model.count_tokens('\n'.join(message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict())).total_tokens
# eg. if factor is 3.2, round to 4
factor = math.ceil(factor)
thread = team.context_threads['all'].copy()
thread *= factor
source_swe_coder.model.count_tokens('\n'.join(message['parts'][0] for interaction in thread for message in interaction.to_dict())).total_tokens

41739

In [19]:
source_swe_coder.cache_context(thread, ttl=datetime.timedelta(minutes=8))

In [20]:
def write(file_path, prompt):
    response = source_swe_coder.chat(prompt)
    md = extract_markdown_blocks(response)[0]
    path = f"{working_dir_path}\\{file_path}"
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    with open(path, 'w') as f:
        f.write(md)

def edit(file_path, prompt):
    with open(f"{working_dir_path}\\{file_path}", 'r') as f:
        content = f.read()
    response = source_swe_coder.chat(f"""{prompt}
Respond in the following format:
# Old Code Snippet
```
$old code snippet to be changed (this string must match exactly with a substring of the original content) (no filler comments etc.)$
```
# New Code Snippet
```
$the code snippet the prompt is asking for$
```""", custom_context=[Interaction(f"Contents of {file_path}", content)])
    old_code, new_code = extract_markdown_blocks(response)
    with open(f'{working_dir_path}\\{file_path}', 'w') as f:
        f.write(content.replace(old_code, new_code))

def delete(file_path):
    os.remove(f"{working_dir_path}\\{file_path}")

In [22]:
from utils.errorutils import repeat_until_finish
from tqdm import tqdm

for action, file, prompt in tqdm(actions):
    if action == 'WRITE':
        repeat_until_finish(lambda: write(file, prompt), max_retries=2)
    elif action == 'DELETE':
        delete(file)
    elif action == 'EDIT':
        repeat_until_finish(lambda: edit(file, prompt), max_retries=2)

 56%|█████▌    | 5/9 [03:31<02:49, 42.32s/it]


ValueError: not enough values to unpack (expected 2, got 1)